In [ ]:
### Import packages
#### Create connection to Database

In [13]:
from sqlalchemy import create_engine, inspect
import pandas as pd

# Connect to the database
db_connection_string = 'sqlite:///chinook.db'
db_engine = create_engine(url=db_connection_string)
db_conn = db_engine.connect()
db_inspector = inspect(db_engine)

#### Read table from database

In [ ]:
# Create an environment 
# python python -m venv .venv (.venv is the environment)

# Activate the environment
# .venv\Scripts\activate

In [ ]:
# HINTS
# Load data into DataFrame
# user Pandas to read data from a table into a DataFram  

In [ ]:
# # Approach 1: Use Pandas.read_sql_table to read all columns from 'customers' table 
# Un # Ctrl + /
query = "SELECT * FROM albums"
df_albums = pd.read_sql_query(query, db_engine)
df_albums.head()

In [14]:
tables = db_inspector.get_table_names()
print("Table found: ", tables)

Table found:  ['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


In [ ]:
dataFrames = {}

for table in tables: 
    df = pd.read_sql_query(f"select * from {table}", db_engine)
    dataFrames[table] = df

# Showing an example
dataFrames["albums"].head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [ ]:
# # Approach 2: Use Pandas.read_sql_query to read these columns
# table_name = 'customers'
# columns = ['CustomerId', 'FirstName', 'LastName', 'Phone', 'Email', 'SupportRepId']
# df = pd.read_sql_query()

#### Config-Driven Ingestion

In [ ]:
# # HINTS
# # Read configs stored in the 'config.yml' file

# # Read yaml file
# # Package: yaml (pip install pyyaml)
# # Function: load / safe_load
# # Print it after loading

# import yaml
# import json

# config_file = 'config.yml'

In [ ]:
# Use loop function to read tables within config.source.table
# Export output into CSV
# Name Convention: '<date>__<table_name>.csv'
# Path: chinook/config_driven/
# note: use os.makedirs() if path is not exists


### Metadata-Driven Ingestion

In [ ]:
# # HINTS
# # Read metadata from the database inlcuding tables / columns
# sqlite_metadata_table = 'sqlite_master'
# sqlite_metadata_condition = "type = 'table'"
# metadata_sql = f""" select 1"""
# print(metadata_sql)
# table_df = pd.read_sql_query(metadata_sql)
# print(table_df)

In [ ]:
# loop for each table from the DataFrame
# read and extract table
# save to path: chinook/metadata_driven/
# note: use os.makedirs() if path is not exists